1. **DATA IMPORT**

I obtained the latest data from the Airbnb website of Amsterdam, North Holland, The Netherlands (3rd September, 2023).

Please refer to this link: http://insideairbnb.com/get-the-data/

The dataset we used is: listings.csv

I started by transforming the untidy data into appropriate formats for analysis, and answering key questions such as:

1. What is the average price per night of an Airbnb listing in Amsterdam?
2. How does the average price of an Airbnb listing per month compare to the private rental market?
3. How many adverts are for private rooms?
4. How do Airbnb listing prices compare across the neighbourhood_cleansed?



In [27]:
import pandas as pd
import numpy as np
import datetime as dt
# for this project we only need "id","host_name","last_review","price","neighbourhood_cleansed","description","room_type" columns.
listings = pd.read_csv("listings.csv",usecols=["id","host_name","last_review","price","neighbourhood_cleansed","description","room_type"])
print(f"listings: {listings.head()}","\n",)

listings:        id                                        description host_name  \
0  761411  Really quiet, spacious and safe, a nice place ...    Xsjong   
1  768274  Our cool and spacious loft is perfect for a st...     J & R   
2  768737  Room to rent in my houseboat. The room has a p...    Nicole   
3  771217  Spacious houseboat in Amsterdam, suitable for ...  Danielle   
4  771343  Royal Bed & Coffee Room with a very comfortabl...    Marcel   

  neighbourhood_cleansed        room_type    price last_review  
0             Noord-Oost     Private room   $61.00  2023-08-19  
1             Westerpark  Entire home/apt  $327.00  2023-08-16  
2             Westerpark     Private room  $109.00  2023-08-24  
3                   Zuid  Entire home/apt  $290.00  2019-01-02  
4           Centrum-West     Private room  $150.00  2023-09-01   



Now let's get the column names and respective data types.
This info helps us in better data handling.

In [28]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8386 entries, 0 to 8385
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      8386 non-null   int64 
 1   description             8385 non-null   object
 2   host_name               8386 non-null   object
 3   neighbourhood_cleansed  8386 non-null   object
 4   room_type               8386 non-null   object
 5   price                   8386 non-null   object
 6   last_review             7556 non-null   object
dtypes: int64(1), object(6)
memory usage: 458.7+ KB


Let's examine the data attribute-wise.

In [29]:
listings.head()

,id,description,host_name,neighbourhood_cleansed,room_type,price,last_review
0,761411,"Really quiet, spacious and safe, a nice place ...",Xsjong,Noord-Oost,Private room,$61.00,2023-08-19
1,768274,Our cool and spacious loft is perfect for a st...,J & R,Westerpark,Entire home/apt,$327.00,2023-08-16
2,768737,Room to rent in my houseboat. The room has a p...,Nicole,Westerpark,Private room,$109.00,2023-08-24
3,771217,"Spacious houseboat in Amsterdam, suitable for ...",Danielle,Zuid,Entire home/apt,$290.00,2019-01-02
4,771343,Royal Bed & Coffee Room with a very comfortabl...,Marcel,Centrum-West,Private room,$150.00,2023-09-01


Check if there are duplicates in dataframe.(if there are any duplicates handle them).

In [30]:
print("There are {} duplicates in the DataFrame.".format(listings.duplicated().sum()))

There are 0 duplicates in the DataFrame.


2. **CLEANING THE PRICE COLUMN**

Having observed that the "price" attribute contains values with dollar symbols and commas.

price

\$61.00

\$327.00

\$109.00

we need to remove these symbols and commas in order to facilitate the use of aggregate functions on the numeric price.



In [31]:
listings["price"] = listings["price"].str.replace("$", "")
listings["price"] = listings["price"].str.replace(",", "")
listings["price"] = pd.to_numeric(listings["price"])
print(listings["price"].head())
print(listings["price"].describe())

0     61.0
1    327.0
2    109.0
3    290.0
4    150.0
Name: price, dtype: float64
count     8386.000000
mean       254.489864
std        395.468275
min         18.000000
25%        150.000000
50%        207.000000
75%        300.000000
max      27857.000000
Name: price, dtype: float64


<ipython-input-31-e025af3f4bc6>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  listings["price"] = listings["price"].str.replace("$", "")


3. **CALCULATING THE AVERAGE PRICE PER NIGHT**

In the dataset, three-quarters of the listings are priced at \$300 per night.

 However, it's important to note the presence of outliers, such as a maximum nightly rate of \$27,857. Conversely, the minimum price per night is as low as \$18.

Now, let's proceed with an analysis of the average nightly price.

In [32]:
avg_price = round(listings["price"].mean(),2)
print("The average price per night for an Airbnb listing in NYC is ${}.".format(avg_price))

The average price per night for an Airbnb listing in NYC is $254.49.


4. **COMPARING COSTS TO THE PRIVATE RENTAL MARKET**

Having obtained the average nightly cost of a listing, it is beneficial to establish a benchmark for comparative analysis. According to Chat GPT, the average monthly cost of a 1-bedroom apartment in New York City is \$2,256. To enable a meaningful comparison with the private market, we should transform the nightly prices of our listings into monthly expenses. This conversion will facilitate a data-driven assessment.

In [33]:
listings["price_per_month"] = listings["price"] * 365 / 12
average_price_per_month = round(listings["price_per_month"].mean(),2)
print("Airbnb monthly costs are ${}, while(according to Chat-GPT)in the private market you would pay {}.".format(average_price_per_month, "$2256.00"))

Airbnb monthly costs are $7740.73, while(according to Chat-GPT)in the private market you would pay $2256.00.


5. **CLEANING THE room_type COLUMN**

Predictably, utilizing Airbnb is evidently associated with considerably higher costs in comparison to the private rental market.

Nevertheless, it's imperative to acknowledge that these Airbnb listings encompass a variety of accommodation types, including single private rooms, shared rooms, and entire homes/apartments.

To gain a more profound understanding, let's perform a detailed analysis of the "room_type" column to discern the distribution of listings based on room types.

It's worth noting that the "room_type" column exhibits variations in the representation of private room listings, specifically "Private room," "private room," and "PRIVATE ROOM."

This can be efficiently addressed by standardizing the text to lowercase, which is a common practice in data preprocessing for uniformity and consistency.



In [34]:
listings["room_type"] = listings["room_type"].str.lower()
listings["room_type"] = listings["room_type"].astype("category")
room_frequencies = listings["room_type"].value_counts()
print(room_frequencies)

entire home/apt    6477
private room       1813
hotel room           54
shared room          42
Name: room_type, dtype: int64


6. **WHAT TIMEFRAME ARE WE WORKING WITH?**

The analysis indicates a market opportunity predominantly focused on homes and apartments, with entire homes/apartments accounting for 76% of listings, followed by private rooms at 22% on the Airbnb platform in NYC.

For the "last_review" column, which contains review dates in the "Year Month Date" format (e.g., 2013-08-19), it is imperative to identify the earliest and latest review dates within the DataFrame.

Additionally, it is essential to format the data in a manner conducive to seamless analysis in the future.

In [35]:
listings["last_review"] = pd.to_datetime(listings["last_review"], format="%Y-%m-%d")
# Drop rows with missing last_review values
listings = listings.dropna(subset=["last_review"])
oldest_review = listings["last_review"].dt.date.min()
latest_review = listings["last_review"].dt.date.max()
print("The latest Airbnb review is {}, the earliest review is {}".format(latest_review, oldest_review))

The latest Airbnb review is 2023-09-03, the earliest review is 2013-08-19


7. **ANALYZING LISTING PRICES BY AMSTERDAM neighbourhood_cleansed**

Let's now shift our focus to comprehending the price disparities among various "neighbourhood_cleansed" categories in the dataset, employing data science terminology for a more precise analysis.

In [36]:
neighbourhood_cleansed = listings.groupby("neighbourhood_cleansed")["price"].agg(["sum", "mean", "median", "count"])
neighbourhood_cleansed = neighbourhood_cleansed.round(2).sort_values("mean", ascending=False)
print(neighbourhood_cleansed)

                                             sum    mean  median  count
neighbourhood_cleansed                                                 
Centrum-West                            282865.0  300.28   238.5    942
Centrum-Oost                            208535.0  285.66   225.0    730
De Pijp - Rivierenbuurt                 226526.0  267.76   238.0    846
Zuid                                    135136.0  264.45   225.0    511
De Baarsjes - Oud-West                  303314.0  241.88   210.0   1254
IJburg - Zeeburgereiland                 39149.0  238.71   200.0    164
Watergraafsmeer                          50652.0  230.24   199.5    220
Oud-Oost                                106015.0  229.97   199.0    461
Westerpark                              120581.0  227.94   200.0    529
Oud-Noord                                75440.0  212.51   189.0    355
Buitenveldert - Zuidas                   15217.0  200.22   185.0     76
Oostelijk Havengebied - Indische Buurt   57246.0  200.16   180.0

In [37]:
# This is to print whole output without truncation.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

8. **PRICE RANGE BY neighbourhood_cleansed**

In this concluding task, our objective is to categorize listings based on specific price ranges within the 'neighbourhood_cleansed'. We shall accomplish this by leveraging percentiles and labels to introduce a novel column, 'price_range,' into the DataFrame.

After the label creation, we will proceed to group the data and compute the frequency of listings falling into distinct price ranges for each 'neighbourhood_cleansed.'

Our categorization will adhere to the following categories and associated price intervals:

**label**	**price**

Budget	    \$0-69

Average	    \$70-175

Expensive	  \$176-350

Extravagant	 > \$350

In [38]:
label_names = ["Budget", "Average", "Expensive", "Extravagant"]
ranges = [0, 69, 175, 350, np.inf]
listings["price_range"] = pd.cut(listings["price"], bins=ranges, labels=label_names)
prices_by_neighbourhood_cleansed = listings.groupby(["neighbourhood_cleansed", "price_range"])["price_range"].agg("count")
print(prices_by_neighbourhood_cleansed)

neighbourhood_cleansed                  price_range
Bijlmer-Centrum                         Budget           6
                                        Average         26
                                        Expensive       11
                                        Extravagant      2
Bijlmer-Oost                            Budget           1
                                        Average         22
                                        Expensive        9
                                        Extravagant      1
Bos en Lommer                           Budget          12
                                        Average        157
                                        Expensive      182
                                        Extravagant     25
Buitenveldert - Zuidas                  Budget           3
                                        Average         29
                                        Expensive       38
                                        Extravagant      6
Cent